In [1]:
import requests
import os

# API endpoint
url = 'https://maps.googleapis.com/maps/api/distancematrix/json'

API_KEY = os.getenv('API_KEY')

# Define the list of origin and destination coordinates
origins = [
    '50.85045,4.34878',  # Grand Place, Brussels
    '51.21944,4.40246',  # Antwerp Central Station
    '51.22172,4.41539',  # Rubenshuis, Antwerp
    # Add more origin coordinates as needed
]

destinations = [
    '50.89504,4.34134',  # Atomium, Brussels
    '51.21291,4.42549',  # MAS Museum, Antwerp
    '51.21762,4.41594',  # Cathedral of Our Lady, Antwerp
    # Add more destination coordinates as needed
]

# API parameters
params = {
    'origins': '|'.join(origins),
    'destinations': '|'.join(destinations),
    'key': API_KEY
}

response = requests.get(url, params=params)

if response.status_code == 200:
    # Parse JSON response
    data = response.json()

    # Extract distance and duration information
    rows = data['rows']
    for row in rows:
        elements = row['elements']
        for element in elements:
            distance = element['distance']['text']
            duration = element['duration']['text']

            # Print information
            print(f"Distance: {distance}")
            print(f"Duration: {duration}")
else:
    print("fail")


Distance: 8.1 km
Duration: 24 mins


In [1]:
data

NameError: name 'data' is not defined

In [24]:
# reading xml input file

import xml.etree.ElementTree as ET

def read_xml_file(xml_file_name):
    tree = ET.parse(xml_file_name)
    root = tree.getroot()
    return root


data = read_xml_file('ulb/trips.trips.xml')

In [25]:
len(data)

3600

In [12]:
!pip install sumolib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 159.3 kB/s eta 0:00:00a 0:00:01


In [21]:
import sumolib

net = sumolib.net.readNet('ulb/ulb.net.xml')

# retrieve the coordinate of a node based on its ID
print (net.getNode('1825074245').getCoord())
print (net.getNode('338336532').getCoord())

# retrieve the successor node ID of an edge
#nextNodeID = net.getEdge('m30601140').getToNode().getID()

(2458.55, 7025.86)
(2453.81, 7028.15)


In [18]:
net.getEdge('30601140#2')

<edge id="30601140#2" from="1825074245" to="338336532"/>

In [26]:
net.getEdge('30089441#2').getShape()

[(4494.31, 7284.06), (4494.43, 7283.89)]

In [ ]:

@function
def getEdgeCoordinates(net, edgeID):
    edge = net.getEdge(edgeID)
    return edge.getShape()